# CS 109A/AC 209A/STAT 121A Data Science: Lab 10 (Solutions)
**Harvard University**<br>
**Fall 2016**<br>
**Instructors: W. Pan, P. Protopapas, K. Rader**<br>
**Due Date: ** Wednesday, November 9th, 2016 at 11:59pm

In [1]:
import matplotlib

import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.cross_validation import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.tree import DecisionTreeClassifier as DecisionTree
from sklearn.ensemble import RandomForestClassifier as RandomForest
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA

%matplotlib inline

/usr/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/usr/local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Predicting Outcome of a Fund-raising Campaign, Revisited

In this lab, we revisit a problem we tackled in Homework 4. 

You are provided a data set containing details of mail sent to 95,412 potential donors for a fund-raising campaign of a not-for-profit organization. This data set also contains the amount donated by each donor. The task is to build a model that can identify which donors to mail in order to expected maximize net contribution, given the cost of mailing a flyer or package is \$7 per donor. 

In Homework 4, we used linear regression to first predict the amount that each individual will donate and then mailed a flyer to only those individuals that were predicted to donate more than \$7.

In this lab, we will cast this problem as a classification problem: we build a model to classify each individual as a mail-worthy donor (will likely donate more than \$7) or a un-mail-worthy donor (will likely donate less than \$7). Again, our goal is to maximize the expected net contribution.

The data is contained in the file `dataset.txt`. Each row contains 376 attributes for a donor, followed by the donation amount.

## Step 1: Clean and explore the data

First let's read and explore the data.

In [20]:
df = pd.read_csv('datasets/dataset.txt')
print df.shape
df.head()

(8928, 377)


,OSOURCE,TCODE,STATE,MAILCODE,NOEXCH,RECINHSE,RECP3,RECPGVG,RECSWEEP,MDMAUD,...,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,CLUSTER2,GEOCODE2,TARGET_D
0,BBK,2.0,MN,_,0,_,_,_,_,XXXX,...,1.0,L,3.0,D,X,X,X,3.0,A,4.0
1,SYN,0.0,TX,_,0,_,_,_,_,XXXX,...,1.0,L,3.0,D,X,X,X,14.0,A,7.0
2,DRK,0.0,IA,_,0,_,_,_,_,XXXX,...,1.0,L,3.0,D,X,X,X,11.0,C,5.0
3,BHG,0.0,CA,_,0,_,_,_,_,XXXX,...,0.0,L,2.0,F,X,X,X,2.0,A,13.0
4,L01,1.0,GA,_,0,_,_,_,_,XXXX,...,1.0,L,3.0,E,X,X,X,22.0,A,10.0


In [21]:
df['MAILCODE'].value_counts()

_    8812
B     116
Name: MAILCODE, dtype: int64

In [22]:
df = df.replace('_', np.nan)
df.head()

,OSOURCE,TCODE,STATE,MAILCODE,NOEXCH,RECINHSE,RECP3,RECPGVG,RECSWEEP,MDMAUD,...,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,CLUSTER2,GEOCODE2,TARGET_D
0,BBK,2.0,MN,NaN,0,NaN,NaN,NaN,NaN,XXXX,...,1.0,L,3.0,D,X,X,X,3.0,A,4.0
1,SYN,0.0,TX,NaN,0,NaN,NaN,NaN,NaN,XXXX,...,1.0,L,3.0,D,X,X,X,14.0,A,7.0
2,DRK,0.0,IA,NaN,0,NaN,NaN,NaN,NaN,XXXX,...,1.0,L,3.0,D,X,X,X,11.0,C,5.0
3,BHG,0.0,CA,NaN,0,NaN,NaN,NaN,NaN,XXXX,...,0.0,L,2.0,F,X,X,X,2.0,A,13.0
4,L01,1.0,GA,NaN,0,NaN,NaN,NaN,NaN,XXXX,...,1.0,L,3.0,E,X,X,X,22.0,A,10.0


In [23]:
df = df.dropna(how = 'all')
df.head()

,OSOURCE,TCODE,STATE,MAILCODE,NOEXCH,RECINHSE,RECP3,RECPGVG,RECSWEEP,MDMAUD,...,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,CLUSTER2,GEOCODE2,TARGET_D
0,BBK,2.0,MN,NaN,0,NaN,NaN,NaN,NaN,XXXX,...,1.0,L,3.0,D,X,X,X,3.0,A,4.0
1,SYN,0.0,TX,NaN,0,NaN,NaN,NaN,NaN,XXXX,...,1.0,L,3.0,D,X,X,X,14.0,A,7.0
2,DRK,0.0,IA,NaN,0,NaN,NaN,NaN,NaN,XXXX,...,1.0,L,3.0,D,X,X,X,11.0,C,5.0
3,BHG,0.0,CA,NaN,0,NaN,NaN,NaN,NaN,XXXX,...,0.0,L,2.0,F,X,X,X,2.0,A,13.0
4,L01,1.0,GA,NaN,0,NaN,NaN,NaN,NaN,XXXX,...,1.0,L,3.0,E,X,X,X,22.0,A,10.0


In [25]:
# No integers, all floats
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8928 entries, 0 to 8927
Columns: 377 entries, OSOURCE to TARGET_D
dtypes: float64(307), object(70)
memory usage: 25.7+ MB


In [33]:
# Drop cols if any NaN
df = df.dropna(axis = 1, how = 'any')
df.head()

,TCODE,STATE,NOEXCH,MDMAUD,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,...,AVGGIFT,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,CLUSTER2,TARGET_D
0,2.0,MN,0,XXXX,10.0,2.0,25.0,40.0,27.0,11.0,...,4.066667,1.0,L,3.0,D,X,X,X,3.0,4.0
1,0.0,TX,0,XXXX,0.0,1.0,37.0,58.0,16.0,8.0,...,6.181818,1.0,L,3.0,D,X,X,X,14.0,7.0
2,0.0,IA,0,XXXX,5.0,0.0,33.0,24.0,39.0,6.0,...,4.857143,1.0,L,3.0,D,X,X,X,11.0,5.0
3,0.0,CA,0,XXXX,0.0,0.0,34.0,20.0,54.0,2.0,...,11.000000,0.0,L,2.0,F,X,X,X,2.0,13.0
4,1.0,GA,0,XXXX,10.0,0.0,21.0,53.0,8.0,5.0,...,9.400000,1.0,L,3.0,E,X,X,X,22.0,10.0


In [37]:
enc = preprocessing.OneHotEncoder()

categoricals = []

for i in df.columns:
    if df[i].dtype == object:
        categoricals.append(i)
        
enc.fit_transform(df[categoricals])

ValueError: could not convert string to float: X

It looks like the NaN values appear only in a few columns. For the sake of expediency, we will simply drop the columns with missing values. **You might want to handle the missing data in a more sophisticated way**.

We see that some of the predictors are categorical and some are quantitative. We need to convert the real-valued predictor values into floating points and encode the categorical variables as integers (we will furthre convert the categorical variables into binary variable using one-hot-encoding). To decide which variables are quantitative and which are categorical, you need to interpret the meaning of each predictor and use your common sense.

Let's split our dataset into train and test.

Recall that the response variable, amount donated, is a real-valued variable not a binary variable! We need to convert the amount donated into a binary value: 0 for under \$7 and 1 for over \$7.

---

## Step 2: Establish the Baseline Models (for Sanity Check)

What are the baseline models in this case? We can check off three basic models: 

1. a model that labels everything 1
2. a model that labels everything 0
3. a model that randomly guesses a label, 1 or 0

Before implementing anything fancy, let's implement these baseline models and see how they do.

**Note:** Again, think about accuracy in a **meaningful** way.

In [9]:
#Function for computing the accuracy a given model on the entire test set, the accuracy on class 0 in the test set
#and the accuracy on class 1
score = lambda model, x_test, y_test: pd.Series([model.score(x_test, y_test), 
                                                 model.score(x_test[y_test==0], y_test[y_test==0]),
                                                 model.score(x_test[y_test==1], y_test[y_test==1])],
                                                index=['overall accuracy', 'accuracy on class 0', 'accuracy on class 1'])

In [10]:
#A model that labels everything 1
class Pos_model(object):
    def predict(self, x):
        return np.array([1] * len(x))
    def score(self, x, y):
        y_pred = self.predict(x)
        y_err = y - y_pred
        return len(y_err[y_err == 0]) * 1. / len(y_err)
    
#A model that labels everything 0
class Neg_model(object):
    def predict(self, x):
        return np.array([0] * len(x))
    
    def score(self, x, y):
        y_pred = self.predict(x)
        y_err = y - y_pred
        return len(y_err[y_err == 0]) * 1. / len(y_err)


#A model that randomly labels things
class Random_model(object):
    def predict(self, x):
        return np.random.randint(0, 2, len(x))
    
    def score(self, x, y):
        y_pred = self.predict(x)
        y_err = y - y_pred
        return len(y_err[y_err == 0]) * 1. / len(y_err)

In [11]:
pos_model = Pos_model()
pos_model_scores = score(pos_model, x_test, y_test)

neg_model = Neg_model()
neg_model_scores = score(neg_model, x_test, y_test)

random_model = Random_model()
random_model_scores = score(random_model, x_test, y_test)

In [12]:
#Score Dataframe
score_df = pd.DataFrame({'pos model': pos_model_scores,
                         'neg model': neg_model_scores,
                         'random model': random_model_scores})
score_df

,neg model,pos model,random model
overall accuracy,0.561882,0.438118,0.493560
accuracy on class 0,1.000000,0.000000,0.519934
accuracy on class 1,0.000000,1.000000,0.497657


---

## Step 3: Build Fancier Models

Now that we have an idea of how baseline models perform, let's try to improve upon them with fancier classifiers.

In [13]:
#Unweighted logistic regression
unweighted_logistic = LogisticRegression()
unweighted_logistic.fit(x_train, y_train)

unweighted_log_scores = score(unweighted_logistic, x_test, y_test)
print 'unweighted log'


#Weighted logistic regression
weighted_logistic = LogisticRegression(class_weight='balanced')
weighted_logistic.fit(x_train, y_train)

weighted_log_scores = score(weighted_logistic, x_test, y_test)
print 'weighted log'

#LDA
lda = LDA()
lda.fit(x_train, y_train)

lda_scores = score(lda, x_test, y_test)
print 'lda'

#QDA
qda = QDA()
qda.fit(x_train, y_train)

qda_scores = score(qda, x_test, y_test)
print 'qda'

#Decision Tree
tree = DecisionTree(max_depth=6)
tree.fit(x_train, y_train)

tree_scores = score(tree, x_test, y_test)
print 'tree'

#Random Forest
rf = RandomForest()
rf.fit(x_train, y_train)

rf_scores = score(rf, x_test, y_test)

print 'rf'

unweighted log
weighted log
lda

//anaconda/lib/python2.7/site-packages/sklearn/discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
//anaconda/lib/python2.7/site-packages/sklearn/discriminant_analysis.py:688: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")



qda
tree
rf


In [14]:
#Score Dataframe
score_df = pd.DataFrame({#'knn': knn_scores, 
                         'unweighted logistic': unweighted_log_scores,
                         'weighted logistic': weighted_log_scores,
                         'lda': lda_scores,
                         'qda': qda_scores,
                         'tree': tree_scores,
                         'rf': rf_scores})
score_df

,lda,qda,rf,tree,unweighted logistic,weighted logistic
overall accuracy,0.567855,0.538921,0.563375,0.566735,0.566175,0.564682
accuracy on class 0,0.657475,0.562458,0.746844,0.748505,0.648505,0.537874
accuracy on class 1,0.452919,0.508735,0.328078,0.333617,0.460588,0.599063


So which model is better? Which out-performs our baseline models? What does "better" mean anyways? 

To perform meaningful model selection, we have to remember our task! We've been asked to create a classifier that will maximize expected net contribution! While accuracy is a helpful metric, it is not clear, by looking at these numbers, which model will generate a greater expected net contribution!

---

## Step 4: Meaningful Model Evaluation

To meaningully assess the effecitveness of our models, we have to evaluate them according to the utility function that computes the expected net contribution.

In [15]:
#--------  expected_profit
# A function that computes the expected net contribution generated from a mailing scheme based on
# your classification of potential doners as mail-worthy or not mail-worthy
# Input: 
#      y_true_val (the true amount donated by each individual)
#      y_true (true class labels)
#      y_pred (predicted class labels)
# Returns: 
#      expected_profit (expected net contribution)

def expected_profit(y_true_val, y_true, y_pred):
    
    profit = []
    
    for i in range(5000):
    
        sample = np.random.choice(len(y_true_val), len(y_true_val))

        true_donations = y_true_val[sample]
        true_labels = y_true[sample]
        pred_labels = y_pred[sample]

        pred_donors = pred_labels > 0

        cost = (pred_donors).sum() * 7.

        donations = true_donations[pred_donors].sum()

        profit.append(donations - cost)
        
    expected_profit = np.mean(profit)
    
    return expected_profit

In the following, we implement a selection of classification models. You should try more (like KNN, SVM, logistic with polynomial decision boundaries etc.)!

In [16]:
profits = []

#Unweighted logistic regression
y_pred = unweighted_logistic.predict(x_test)

profits.append(expected_profit(y_test_val, y_test, y_pred))

print 'unweighted log'

#Weighted logistic regression
y_pred = weighted_logistic.predict(x_test)

profits.append(expected_profit(y_test_val, y_test, y_pred))

print 'weighted log'

#LDA
y_pred = lda.predict(x_test)

profits.append(expected_profit(y_test_val, y_test, y_pred))

print 'lda'

#QDA
y_pred = qda.predict(x_test)

profits.append(expected_profit(y_test_val, y_test, y_pred))

print 'qda'

#Decision Tree
y_pred = tree.predict(x_test)

profits.append(expected_profit(y_test_val, y_test, y_pred))

print 'tree'

#Random Forest
y_pred = rf.predict(x_test)

profits.append(expected_profit(y_test_val, y_test, y_pred))

print 'rf'

#Positive Baseline Model
y_pred = pos_model.predict(x_test)

profits.append(expected_profit(y_test_val, y_test, y_pred))

print 'pos baseline'

#Negative Baseline Model
y_pred = neg_model.predict(x_test)

profits.append(expected_profit(y_test_val, y_test, y_pred))

print 'neg baseline'

#Random Baseline Model
y_pred = random_model.predict(x_test)

profits.append(expected_profit(y_test_val, y_test, y_pred))

print 'rand baseline'

#Total possible profit (if all predictions are accurate)
profits.append(expected_profit(y_test_val, y_test, y_test))

print 'total possible profit'

unweighted log
weighted log
lda
qda
tree
rf
pos baseline
neg baseline
rand baseline
total possible profit


In [17]:
#Score Dataframe
score_df = pd.DataFrame(data=np.array(profits).reshape((1, len(profits))), index=['net contribution'], columns=['unweighted log', 'weighted log', 'lda', 'qda', 'tree', 'rf', 'pos', 'neg', 'random', 'max possible profit'])
score_df

,unweighted log,weighted log,lda,qda,tree,rf,pos,neg,random,max possible profit
net contribution,5414.2376,6868.6997,5098.57895,5337.559,3273.21105,3282.83162,8145.186588,0,4055.71437,26349.864604


How did each model do - which is the best model? 

Compare the net contribution obtained from each model with the accuracy of each model. How is accuracy related to the net contribution? Is accuracy a good predictor of net contribution (meaning is accuracy a meaningful metric for evaluating our models)?

What can we do to improve the performance of our models? 

**Hint:** Would tuning these models help? Consider, also, all the ensemble methods you've explored in the past two Homework sets: boosting and meta-learning.

---

## Step 5: Tuning to Maximize Net Contribution

Take any model from the above, we can try to improve its performance (in terms of maximizing net contribution) by tuning its parameters to maximize the profit function. Here we demonstrate how to tune a decision tree. You should try the same for the rest of the models.

In [18]:
depths = range(1, 20)
kf = KFold(len(x_train), n_folds=5)
profits = []

for depth in depths:
    print 'depth:', depth
    validation_profits = []
    for train_index, test_index in kf:
        x_validate_train, x_validate_test = x_train[train_index], x_train[test_index]
        y_validate_train, y_validate_test = y_train[train_index], y_train[test_index]
        
        tree = DecisionTree(max_depth=depth)
        tree.fit(x_validate_train, y_validate_train)
        
        y_pred = tree.predict(x_validate_test)
        validation_profits.append(expected_profit(y_test_val[test_index], y_validate_test, y_pred))
        
    profits.append(np.mean(validation_profits))

best_depth = np.argmax(profits) + 1

tree = DecisionTree(max_depth=best_depth)
tree.fit(x_train, y_train)
y_pred = tree.predict(x_test)

print 'profits for tuned tree with depth {}: {}'.format(best_depth, expected_profit(y_test_val, y_test, y_pred))

depth: 1
depth: 2
depth: 3
depth: 4
depth: 5
depth: 6
depth: 7
depth: 8
depth: 9
depth: 10
depth: 11
depth: 12
depth: 13
depth: 14
depth: 15
depth: 16
depth: 17
depth: 18
depth: 19
profits for tuned tree with depth 18: 4838.32098


Did the tuning improve the performance of our tree model?

Now, consider what you can tune for weighted logistic regression (or SVM or random forest). Certainly, there is the regularization parameter $C$. In the above we just weighted each sample by the reciprocal of the class proportion (for the class that the sample belongs to). But can we weight the samples differently (using the `sample_weight` parameter in the `.fit()` function of the `sklearn` models)? How can you weight the samples to achieve maximum profit?

**Hint:** think about how profit is related to accuracy in this application.